Import modules and create spark session

In [1]:
#import modules
import pyspark
from pyspark.sql.types import *
from pyspark.sql.functions import *
from pyspark.ml.regression import LinearRegression
from pyspark.ml.feature import VectorAssembler
  
#create Spark session
appName = "Regression in Spark"
spark = SparkSession \
    .builder \
    .appName(appName) \
    .config("spark.some.config.option", "some-value")\
    .getOrCreate()

Read data file into Spark dataFrame

In [2]:
#define our schema
flightSchema = StructType([
  StructField("DayofMonth", IntegerType(), False),
  StructField("DayOfWeek", IntegerType(), False),
  StructField("Carrier", StringType(), False),
  StructField("OriginAirportID", IntegerType(), False),
  StructField("DestAirportID", IntegerType(), False),
  StructField("DepDelay", IntegerType(), False),
  StructField("ArrDelay", IntegerType(), False),
])
#read csv data with our defined schema
flightDataFrame = spark.read.csv('dataset/flights.csv', 
                                 schema=flightSchema, header=True)
flightDataFrame.show(3)

+----------+---------+-------+---------------+-------------+--------+--------+
|DayofMonth|DayOfWeek|Carrier|OriginAirportID|DestAirportID|DepDelay|ArrDelay|
+----------+---------+-------+---------------+-------------+--------+--------+
|        19|        5|     DL|          11433|        13303|      -3|       1|
|        19|        5|     DL|          14869|        12478|       0|      -8|
|        19|        5|     DL|          14057|        14869|      -4|     -15|
+----------+---------+-------+---------------+-------------+--------+--------+
only showing top 3 rows



Select important data for regression features

In [3]:
#select related column data for our regression input features
data = flightDataFrame.select("DayofMonth", "DayOfWeek", 
                              "OriginAirportID", "DestAirportID", 
                              "DepDelay", "ArrDelay")
data.show(3)

+----------+---------+---------------+-------------+--------+--------+
|DayofMonth|DayOfWeek|OriginAirportID|DestAirportID|DepDelay|ArrDelay|
+----------+---------+---------------+-------------+--------+--------+
|        19|        5|          11433|        13303|      -3|       1|
|        19|        5|          14869|        12478|       0|      -8|
|        19|        5|          14057|        14869|      -4|     -15|
+----------+---------+---------------+-------------+--------+--------+
only showing top 3 rows



In [4]:
data.columns

['DayofMonth',
 'DayOfWeek',
 'OriginAirportID',
 'DestAirportID',
 'DepDelay',
 'ArrDelay']

In [6]:
for c in data.columns:
    for c1 in data.columns:
        if c!=c1:
            correlation = data.corr(c, c1)
            print("correlation between",c," and ", c1, ":",correlation)


correlation between DayofMonth  and  DayOfWeek : 0.02300430744918717
correlation between DayofMonth  and  OriginAirportID : -0.0006232493298116751
correlation between DayofMonth  and  DestAirportID : -0.0006067902629124677
correlation between DayofMonth  and  DepDelay : 0.006389624175000885
correlation between DayofMonth  and  ArrDelay : 0.008210643961196164
correlation between DayOfWeek  and  DayofMonth : 0.02300430744918717
correlation between DayOfWeek  and  OriginAirportID : 0.0033745625126068296
correlation between DayOfWeek  and  DestAirportID : 0.003230760924124041
correlation between DayOfWeek  and  DepDelay : -0.005670744852869089
correlation between DayOfWeek  and  ArrDelay : -0.012355649745364313
correlation between OriginAirportID  and  DayofMonth : -0.000623249329811673
correlation between OriginAirportID  and  DayOfWeek : 0.0033745625126068326
correlation between OriginAirportID  and  DestAirportID : 0.004527235123535592
correlation between OriginAirportID  and  DepDelay 

Divide data into training and testing data

In [7]:
#divide data, 70% for training, 30% for testing
dividedData = data.randomSplit([0.7, 0.3]) 
trainingData = dividedData[0] #index 0 = data training
testingData = dividedData[1] #index 1 = data testing
train_rows = trainingData.count()
test_rows = testingData.count()
print ("Training data rows:", train_rows, "; Testing data rows:", test_rows)

Training data rows: 1891918 ; Testing data rows: 810300


Prepare training data

In [9]:
#define an assembler
assembler = VectorAssembler(
    inputCols = ["DayofMonth", "DayOfWeek", "OriginAirportID", "DestAirportID", "DepDelay"], 
    outputCol="features")
#change our features into one column using our defined assembler
trainingDataFinal = assembler.transform(trainingData)
trainingDataFinal=trainingDataFinal.select("features",col("ArrDelay").alias("label"))
trainingDataFinal.show(truncate=False , n=3)

+------------------------------+-----+
|features                      |label|
+------------------------------+-----+
|[1.0,1.0,10140.0,10397.0,-2.0]|-17  |
|[1.0,1.0,10140.0,11259.0,-3.0]|-11  |
|[1.0,1.0,10140.0,11259.0,-2.0]|-14  |
+------------------------------+-----+
only showing top 3 rows



Train our regression model using training data

In [10]:
#call Spark linear regression we import before
algoritma = LinearRegression(
    labelCol="label",featuresCol="features", 
    maxIter=10, regParam=0.3)
#train the model
model = algoritma.fit(trainingDataFinal)
print ("Regression model is trained!")

Regression model is trained!


In [12]:
# Print the coefficients and intercept for linear regression
print("Coefficients: %s" % str(model.coefficients))
print("Intercept: %s" % str(model.intercept))

Coefficients: [0.009267807112898912,-0.13541383163944676,0.00019982259941295815,-0.00022451490523063387,0.9971904384101201]
Intercept: -3.1251613730480905


In [16]:
print("RMSE: %f" % model.summary.rootMeanSquaredError)
print("r2: %f" % model.summary.r2)


RMSE: 13.231824
r2: 0.882198


In [18]:
len(model.summary.pValues)

6

In [21]:
print(f'p-values:{model.summary.pValues}')
print(f't-values:{model.summary.tValues}')

p-values:[0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
t-values:[8.509433994855785, -28.057962433844338, 31.315342628059874, -35.1910649502869, 3748.900927781082, -26.519917589360148]


Prepare testing data

In [22]:
#change our feature data into one column using our defined assembler
#just like what we did before in the training data
testingDataFinal = assembler.transform(
    testingData).select(
    col("features"), (col("ArrDelay")).cast("Int").alias("trueLabel"))
testingDataFinal.show(truncate=False, n=2)

+------------------------------+---------+
|features                      |trueLabel|
+------------------------------+---------+
|[1.0,1.0,10140.0,10397.0,-4.0]|-11      |
|[1.0,1.0,10140.0,10821.0,8.0] |-9       |
+------------------------------+---------+
only showing top 2 rows



Predict the testing data using our trained model

In [23]:
#predict testing data using our model
prediction = model.transform(testingDataFinal)
#show some prediction results
prediction.show(3)

+--------------------+---------+-------------------+
|            features|trueLabel|         prediction|
+--------------------+---------+-------------------+
|[1.0,1.0,10140.0,...|      -11| -7.548149462850624|
|[1.0,1.0,10140.0,...|       -9|  4.322941478253028|
|[1.0,1.0,10140.0,...|      -12|-3.7529195575189496|
+--------------------+---------+-------------------+
only showing top 3 rows



Calculate our model performance

In [25]:
#import evaluator module for regression
from pyspark.ml.evaluation import RegressionEvaluator

#define our evaluator
evaluator1 = RegressionEvaluator(
    labelCol="trueLabel", predictionCol="prediction", metricName="rmse")
evaluator2 = RegressionEvaluator(
    labelCol="trueLabel", predictionCol="prediction", metricName="r2")
#calculate RMSE of our trained model
rmse = evaluator1.evaluate(prediction)
r2 = evaluator2.evaluate(prediction)
print ("Root Mean Square Error (RMSE):", rmse)
print ("R^2:", r2)

Root Mean Square Error (RMSE): 13.198976599102487
R^2: 0.8826997265208363
